# 2.8: 量子哈密顿量基态计算

量子哈密顿量$\hat{H}$：描述量子系统"力学"(mechanism)的核心算符。

> 力学指使用数序描述物理系统的演化规律

(定态)薛定谔方程：

$\hat{H}|\varphi\rangle = E|\varphi\rangle$

其中 $E = \langle\varphi|\hat{H}|\varphi\rangle$ 为能量，上述方程可化为本征问题。

> $E$ 的推导:
> 
> $\hat{H}|\varphi\rangle = E|\varphi\rangle$ 两边同乘 $\langle\varphi|$，得到 $E = \langle\varphi|\hat{H}|\varphi\rangle$，根据量子态的归一化性质 $\langle\varphi|\varphi\rangle = 1$，可得 $E = \langle\varphi|\hat{H}|\varphi\rangle$。

> $E$ 也是 $\hat{H}$ 的量子期望

最关心最小（代数）本征值问题——**基态**(ground state)问题：

$|\varphi_{GS}\rangle = \text{argmin}_{|\varphi\rangle}\langle\varphi|\hat{H}|\varphi\rangle$

> 关心最小代数本征值的原因：
>
> 本征值对应能量，能量越小，这个态出现的概率越大。温度越低，态的概率的差距越大。当温度趋近于绝对零度时，基态的概率趋近于1，其他态的概率趋近于0。也就是基态是最稳定的态。

TODO: 看书里 3.1 - 3.4 节对于基态的概率意义的解释

## 例子：4 qubit 的一维海森堡模型

4个自旋（量子比特）构成的一维海森堡模型

$\hat{H} = \frac{1}{4}\sum_{n=0}^2 \hat{H}_{n,n+1}$

$= \frac{1}{4}\sum_{n=0}^2 (\hat{\sigma}_n^x\hat{\sigma}_{n+1}^x + \hat{\sigma}_n^y\hat{\sigma}_{n+1}^y + J\hat{\sigma}_n^z\hat{\sigma}_{n+1}^z)$

哈密顿量$\hat{H}$中的每一项 $\hat{H}_{n,n+1} = \hat{\sigma}_n^x\hat{\sigma}_{n+1}^x + \hat{\sigma}_n^y\hat{\sigma}_{n+1}^y + J\hat{\sigma}_n^z\hat{\sigma}_{n+1}^z$ 为二体算符，其系数为一个 $(4\times4)$ 的矩阵，作用于多体态的规则见2.5节。

> $J$ 叫各项异性参数
>
> 在这个例子里，如果 $J=0$ 那么只有 xy 相互作用
> 
> 如果 $J\gg 1$ 那么只有 z 相互作用，xy 相互作用可以忽略

哈密顿量本身的系数为$(2^4\times2^4)$的矩阵。

### 方法一：算出哈密顿量($2^4 \times 2^4$)的系数的矩阵

$\hat{H}_{01} \rightarrow \text{kron}(\hat{H}_{01},\hat{I},\hat{I})$

$\hat{H}_{12} \rightarrow \text{kron}(\hat{I},\hat{H}_{12},\hat{I})$

$\hat{H}_{23} \rightarrow \text{kron}(\hat{I},\hat{I},\hat{H}_{23})$

图示如下
![hamiltonian_example](./images/hamiltonian_example.png)

相加后使用`eigsh`等函数求解最小本征态
> 计算复杂度：$O(2^N \times 2^N)$

### 方法二：线性算子法

**关键步骤为定义线性映射**

$f(|\varphi\rangle) = \frac{1}{4}\sum_{n=0}^2 \hat{H}_{n,n+1}|\varphi\rangle$

> $\hat{H}$ 是 $4\times4$ 矩阵，因为是二体算符
>
> $|\varphi\rangle$ 是 $4\times4$ 矩阵，因为左指标是 2 qubit 态展开，右指标是剩下的 qubit 态展开

降低了指数级复杂度：对于上述例子，共需进行3次(4×4)与(4×4)间的矩阵乘法，计算复杂度为
$O(3\times4\times4\times4)$

对于长度为N的自旋链，计算复杂度为
$O((N-1)\times4\times4\times2^{N-2})\sim O(N2^N)$

代码例子
![linear_op_hamiltonian_example](./images/linear_op_hamiltonian_example.png)

TODO: 看代码例子